In [33]:
import calliope
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import winsound as ws

In [34]:
hydrogen_costs = np.arange(0,1,0.01) #range of hydrogen costs, to test the cost competitiveness of Green hydrogen production method

time_period = "default"

scenarios = {"default":{"cs1":"default/concept_scenario_1","cs2":"default/concept_scenario_2","cs3":"default/concept_scenario_3","cs4":"default/concept_scenario_4"},
             "high_wind_month":{"cs1":"high_wind_month/concept_scenario_1","cs2":"high_wind_month/concept_scenario_2","cs3":"high_wind_month/concept_scenario_3","cs4":"high_wind_month/concept_scenario_4"},
             "low_wind_month":{"cs1":"low_wind_month/concept_scenario_1","cs2":"low_wind_month/concept_scenario_2","cs3":"low_wind_month/concept_scenario_3","cs4":"low_wind_month/concept_scenario_4"},
             "high_wind_day":{"cs1":"high_wind_day/concept_scenario_1","cs2":"high_wind_day/concept_scenario_2","cs3":"high_wind_day/concept_scenario_3","cs4":"high_wind_day/concept_scenario_4"},
             "low_wind_day":{"cs1":"low_wind_day/concept_scenario_1","cs2":"low_wind_day/concept_scenario_2","cs3":"low_wind_day/concept_scenario_3","cs4":"low_wind_day/concept_scenario_4"},
             "full_year":{"cs1":"full_year/concept_scenario_1","cs2":"full_year/concept_scenario_2","cs3":"full_year/concept_scenario_3","cs4":"full_year/concept_scenario_4"},
            }
                       

In [35]:
def cost_iteration(scenario,cost_range):

    results =[]
    models ={}
    
    for cost in cost_range:
        
        model = calliope.Model('../model.yaml',
                               scenario=scenario,
                               override_dict = {'techs.smr.costs.monetary.om_prod': cost})

        model.run(force_rerun=True)
        
        models[str(cost)]= model
                       
        
        if "concept_scenario_2" in model.results.scenario:
            
            results.append({'hydrogen_cost' : cost,
                            'Green_Hydrogen_prod' : model.results.carrier_prod.sel(loc_tech_carriers_prod='region1-1::electrolysis::hydrogen').values.sum(),
                            'Grey_Hydrogen_prod' : model.results.carrier_prod.sel(loc_tech_carriers_prod='region2::smr::hydrogen').values.sum()
                           }) # store results in list
        else:
            
            results.append({'hydrogen_cost' : cost,
                            'Green_Hydrogen_prod' : model.results.carrier_prod.sel(loc_tech_carriers_prod='region1-2::electrolysis::hydrogen').values.sum(),
                            'Grey_Hydrogen_prod' : model.results.carrier_prod.sel(loc_tech_carriers_prod='region2::smr::hydrogen').values.sum()
                            }) # store results in list
    
    results_df = pd.DataFrame(results)
    
    return results_df, models

In [36]:
[cs1_data, cs1_model] = cost_iteration(scenarios[time_period]["cs1"],hydrogen_costs)
ws.Beep(3000,10) #to let user know it's finished running

In [37]:
[cs2_data, cs2_model] = cost_iteration(scenarios[time_period]["cs2"],hydrogen_costs)
ws.Beep(3000,10) #to let user know it's finished running

In [38]:
[cs3_data, cs3_model] = cost_iteration(scenarios[time_period]["cs3"],hydrogen_costs)
ws.Beep(3000,10) #to let user know it's finished running

In [39]:
[cs4_data, cs4_model] = cost_iteration(scenarios[time_period]["cs4"],hydrogen_costs)
ws.Beep(3000,10) #to let user know it's finished running

ApplicationError: Solver (cbc) did not exit normally

In [ ]:
cs1_data.plot(x="hydrogen_cost")
cs2_data.plot(x="hydrogen_cost")
cs3_data.plot(x="hydrogen_cost")
#cs4_data.plot(x="hydrogen_cost")

In [ ]:
print(cs1_data)

In [ ]:
cs1_model["0.1"].plot.timeseries(array=['hydrogen', 'resource_prod'])